In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 7
model = Network('model')

In [3]:
# Create network
delay = Delay(model,'Delay')
fork1 = Fork(model,'Fork1')
join1 = Join(model,'Join1',fork1)
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)

jobclass1 = ClosedClass(model, 'class1', 1, delay, 0)
jobclass2 = ClosedClass(model, 'class2', 1, delay, 0)

In [4]:
# Service configurations
delay.set_service(jobclass1, Exp(0.25))
queue1.set_service(jobclass1, Exp(2.0))
queue2.set_service(jobclass1, Exp(2.0))

delay.set_service(jobclass2, Exp(0.25))
queue1.set_service(jobclass2, Exp(2.0))
queue2.set_service(jobclass2, Exp(2.0))

In [5]:
# Set routing matrix with class switching
P = model.init_routing_matrix()

# Class 1 routing
P.set(jobclass1, jobclass1, delay, fork1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join1, 1.0)
P.set(jobclass1, jobclass1, queue2, join1, 1.0)
P.set(jobclass1, jobclass1, join1, delay, 1.0)

# Class 2 routing (note the class switching from jobclass2 to jobclass1)
P.set(jobclass2, jobclass2, delay, fork1, 1.0)
P.set(jobclass2, jobclass1, fork1, queue1, 1.0)
P.set(jobclass2, jobclass1, fork1, queue2, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverMVA(model), SolverJMT(model, seed=23000)] # JMT has a bug on this one
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.056588s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   class1  1.6364  1.6364  4.0000  4.0000  0.4091  0.4091
2   Join1   class1  0.2464  0.0000  0.3012  0.3012  1.2273  0.4091
4  Queue1   class1  0.2464  0.2045  0.6024  0.3012  0.6137  0.4090
6  Queue2   class1  0.2464  0.2045  0.6024  0.3012  0.6137  0.4090
SOLVER 2: SolverJMT
JMT Model: /tmp/workspace/jsim/5934446376259706483/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.713442s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0   Delay   class1  0.8324  0.8324  4.0033  4.0033  0.2054  0.2078
2   Join1   class1  0.2241  0.0000  0.2740  0.2740  0.8255  0.2058
4  Queue1   class1  0.2234  0.2007  0.5470  0.2735  0.4146  0.4146
6  Queue2   class1  0.2159  0.1980  0.5504  0.2752  0.4146  0.4146
